# Análise exploratória de dados

A análise exploratória de dados é uma etapa fundamental para qualquer projeto que envolva dados. Ela consiste em examinar e estudar as características de um conjunto de dados, como sua distribuição, suas variáveis, suas relações e suas anomalias, antes de aplicar técnicas mais avançadas de estatística ou aprendizagem de máquina. O objetivo da análise exploratória de dados é obter uma compreensão mais profunda dos dados, identificar padrões, tendências e insights que possam orientar a tomada de decisão e a solução de problemas. A análise exploratória de dados pode ser realizada de forma visual, usando gráficos, tabelas e diagramas, ou de forma numérica, usando medidas de tendência central, dispersão, correlação e teste de hipóteses. A escolha dos métodos depende do tipo e da qualidade dos dados disponíveis, bem como da pergunta ou objetivo que se quer responder com a anál

<!-- <hr style="border-width: 1px" width="95%" > -->
<div></div> 

In [1]:
# Importa os módulos necessários
import numpy as np   # Módulo para trabalhar com matrizes e funções matemáticas
import pandas as pd  # Módulo para trabalhar com dataframes e séries em Python



<div></div> 

## Estruturação dos Arquivos

<div></div> 

In [2]:
# Set the directory path to where the data is stored
doc_dir = '../data/emails/mini_newsgroups/'

# Create an empty list to hold the processed data
database = []

# Iterate over each file in the directory and its subdirectories
for filepath in os.listdir(doc_dir): 
    for filename in os.listdir(f'{doc_dir}{filepath}'):

        # Open each file individually and read its contents
        with open(os.path.join(doc_dir, filepath, filename), 'r') as f:
            text_data = f.read().strip()
        
        # Split the header and body of the email
        try:
            header, body = text_data.split('\n\n', maxsplit=1)
        except:
            continue
        
        # Convert header to a dictionary
        header_dict = {}
        for line in header.split('\n'):
            try:
                # Split the key and value in each header field and store them in a dictionary
                key, value = line.strip().split(': ', maxsplit=1)
                header_dict[key] = value
            except:
                # If a header field cannot be split properly, skip it and continue
                continue
        
        # Append the processed data to the list
        database.append({'filepath': filepath, 
                        'filename': filename,
                        'text': body, 
                        **header_dict
                        })

# tranformation from dict -> dataframe
base_inicial = pd.DataFrame(database)

# remove database from memory
del database
gc.collect()

0

<div></div> 

## Processamento de Texto

<div></div> 

### Transformação de minúsculos

<div></div> 

In [3]:
# Substituindo todos os caracteres que não são letras ou números por espaços em branco, exceto as barras invertidas (\\)
base_inicial['text'] = base_inicial['text'].replace(r'(\\[a-z])|([^\w\\])', ' ', regex=True)

# Aplicando as funções str.lower() e str.strip() simultaneamente
base_inicial['text'] = base_inicial['text'].apply(lambda x: x.lower().strip())


In [4]:
# Faz o download do recurso 'stopwords' do nltk
nltk.download('stopwords')

# Define a lista de stopwords em inglês usando o módulo stopwords do nltk
stopwords = stopwords.words('english')

# Aplica a função lambda em cada linha da coluna 'text' da tabela 'base_inicial'
# A função lambda realiza a tokenização do texto, transforma as palavras em minúsculas e remove as stopwords
base_inicial['text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       article n4hy 93apr5120934 harder ccr p ida org...
1       kmr4 po cwru edu keith ryan writes people keep...
2       livesey solntze wpd sgi com jon livesey writes...
3       bobbe vice ico tek com robert beauchaine write...
4       article 16ba1e927 drporter suvm syr edu drport...
                              ...                        
1994    article 1rgmjn 567 access digex net huston acc...
1995    started reading newsgroup following thread cur...
1996    blessed hunger thirst righteousness filled mat...
1997    curious know christians ever read books based ...
1998    article 1993apr27 073723 18577 csis dit csiro ...
Name: text, Length: 1999, dtype: object

In [5]:
nltk.download('wordnet')    # faz o download do recurso 'wordnet' do nltk
nltk.download('punkt')     # faz o download do recurso 'punkt' do nltk

# Cria um objeto 'w_tokenizer' da classe 'WhitespaceTokenizer' do nltk para tokenizar o texto por espaços em branco
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

# Cria um objeto 'lemmatizer' da classe 'WordNetLemmatizer' do nltk para realizar a lematização das palavras
lemmatizer = nltk.WordNetLemmatizer()

# Define a função 'lemmatizer_text' que recebe um texto como entrada, tokeniza o texto em palavras e lematiza cada palavra
def lemmatizer_text(text): 
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

# Cria uma nova coluna 'tokens' na tabela 'base_inicial' que contém uma lista de tokens lematizados para cada texto
base_inicial['tokens'] = base_inicial['text'].map(lemmatizer_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [13]:
# Cria uma lista de palavras a partir da lista de tokens
w = [j for i in list(itertools.chain(base_inicial['tokens'])) for j in i]

# Instancia um objeto SpellChecker para correção ortográfica
spell = SpellChecker()

# Cria um dicionário com as palavras únicas da lista, faz a correção ortográfica e associa com a palavra original
spell_checked = {word: spell.correction(word) for word in pd.Series(w).unique()}

# Define o caminho do arquivo que irá armazenar o dicionário serializado
path = '../references/spellcheck.pickle'

# Abre o arquivo para gravação em modo binário e escreve o objeto serializado
with open(path, 'wb') as file: 
    pickle.dump(spell_checked, file)

### Export da base

In [28]:
'n'.lower() in ['s', 'y']

False

In [31]:
path = '../data/processed/base_processed.parquet.gzip'

if os.path.isfile(path): 
    answer = input('File already exists, do you want to overwrite? (y/n)')
    if answer.lower() in ['s', 'y']:
        base_inicial.to_parquet(path, compression='gzip')
    else:
        raise FileExistsError('File already exists')
else: 
    base_inicial.to_parquet(path, compression='gzip')

FileExistsError: File already exists